## 任务8： 【选学】1 散点图 (6)：使用逻辑回归探索未知标签



https://www.bilibili.com/video/BV195411778a?p=7&spm_id_from=pageDriver

In [44]:

import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt  # 绘制二维图
import seaborn as sns # 
%matplotlib inline  
# 魔法命令
#如果你在使用Jupyter Notebook，你会需要这样一句命令来让你的图像显示


print(np.__version__)
print(pd.__version__)
print(mpl.__version__)
print(sns.__version__)


#导入数据
midwest = pd.read_csv("https://raw.githubusercontent.com/selva86/datasets/master/midwest_filter.csv")

#提取标签中的类别
print(midwest['category'])
categories = np.unique(midwest['category']) # 去除所有重复的项目
# print(categories) #这就是我们要使用的标签的类别列表

1.19.2
1.1.5
3.3.4
0.11.1
0      AAR
1      LHR
2      AAR
3      ALU
4      AAR
      ... 
327    AAR
328    AAR
329    AAR
330    AAR
331    AAR
Name: category, Length: 332, dtype: object


In [45]:
midwest['category'].value_counts()

AAR    186
LAR     30
LHR     27
AAU     21
AHR     16
ALU     14
ALR     11
HLU     10
HAR      6
HAU      3
LAU      3
LHU      2
HLR      2
HHR      1
Name: category, dtype: int64

In [46]:
# 插入三列
midwest["c1"] = midwest["category"].apply(lambda x:x[0]) # 切片
midwest["c2"] = midwest["category"].apply(lambda x:x[1])
midwest["c3"] = midwest["category"].apply(lambda x:x[2])

# 查看
midwest["c1"].value_counts() # average high, low
# midwest.iloc[:,-3:].head() # 显示后三列  iloc根据行列， loc是根据表头

A    248
L     62
H     22
Name: c1, dtype: int64

In [47]:
# 逻辑回归 先处理数据集，将我们取出来的三列标签转化为数字

from sklearn.preprocessing import OrdinalEncoder as OE # 预处理
data = midwest.copy() # 防止更改原始数据
data.head()

,PID,county,state,area,poptotal,popdensity,popwhite,popblack,popamerindian,popasian,...,percbelowpoverty,percchildbelowpovert,percadultpoverty,percelderlypoverty,inmetro,category,dot_size,c1,c2,c3
0,561,ADAMS,IL,0.052,66090,1270.961540,63917,1702,98,249,...,13.151443,18.011717,11.009776,12.443812,0,AAR,250.944411,A,A,R
1,562,ALEXANDER,IL,0.014,10626,759.000000,7054,3496,19,48,...,32.244278,45.826514,27.385647,25.228976,0,LHR,185.781260,L,H,R
2,563,BOND,IL,0.022,14991,681.409091,14477,429,35,16,...,12.068844,14.036061,10.852090,12.697410,0,AAR,175.905385,A,A,R
3,564,BOONE,IL,0.017,30806,1812.117650,29344,127,46,150,...,7.209019,11.179536,5.536013,6.217047,1,ALU,319.823487,A,L,U
4,565,BROWN,IL,0.018,5836,324.222222,5264,547,14,5,...,13.520249,13.022889,11.143211,19.200000,0,AAR,130.442161,A,A,R


In [48]:
data.iloc[:, -3:].head()

,c1,c2,c3
0,A,A,R
1,L,H,R
2,A,A,R
3,A,L,U
4,A,A,R


In [49]:
data.iloc[:,-3:] = OE().fit_transform(data.iloc[:,-3:]) # label转化为数字形式
data.iloc[:,-3:].head() #average = 0,L=2,H=1

,c1,c2,c3
0,0.0,0.0,0.0
1,2.0,1.0,0.0
2,0.0,0.0,0.0
3,0.0,2.0,1.0
4,0.0,0.0,0.0


In [50]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 332 entries, 0 to 331
Data columns (total 32 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   PID                   332 non-null    int64  
 1   county                332 non-null    object 
 2   state                 332 non-null    object 
 3   area                  332 non-null    float64
 4   poptotal              332 non-null    int64  
 5   popdensity            332 non-null    float64
 6   popwhite              332 non-null    int64  
 7   popblack              332 non-null    int64  
 8   popamerindian         332 non-null    int64  
 9   popasian              332 non-null    int64  
 10  popother              332 non-null    int64  
 11  percwhite             332 non-null    float64
 12  percblack             332 non-null    float64
 13  percamerindan         332 non-null    float64
 14  percasian             332 non-null    float64
 15  percother             3

In [51]:
data.columns = ["城市ID","郡","州","面积","总人口","人口密度","白人人口","非裔人口","美洲印第安人人口","亚洲人口","其他人种人口"
                   ,"白人所占比例","非裔所占比例","美洲印第安人所占比例","亚洲人所占比例","其他人种比例"
                   ,"成年人口","具有高中文凭的比率","大学文凭比例","有工作的人群比例"
                   ,"已知贫困人口","已知贫困人口的比例","贫困线以下的人的比例","贫困线以下的儿童所占比例","贫困的成年人所占的比例","贫困的老年人所占的比例"
                   ,"是否拥有地铁","标签","点的尺寸","c1","c2","c3"]

#去掉所有类型为”object"的列，都是无用的信息=====================================================
data = data.loc[:,data.dtypes.values != "O"]

In [52]:
#将所有为整数类型的类转换为浮点数类型
for i in range(data.loc[:,data.dtypes.values == "int64"].shape[1]):
    data.loc[i,data.dtypes.values == "int64"] = data.loc[i,data.dtypes.values == "int64"].apply(lambda x:float(x))

data.info() # 已经全部转化为float64

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 332 entries, 0 to 331
Data columns (total 29 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   城市ID          332 non-null    float64
 1   面积            332 non-null    float64
 2   总人口           332 non-null    float64
 3   人口密度          332 non-null    float64
 4   白人人口          332 non-null    float64
 5   非裔人口          332 non-null    float64
 6   美洲印第安人人口      332 non-null    float64
 7   亚洲人口          332 non-null    float64
 8   其他人种人口        332 non-null    float64
 9   白人所占比例        332 non-null    float64
 10  非裔所占比例        332 non-null    float64
 11  美洲印第安人所占比例    332 non-null    float64
 12  亚洲人所占比例       332 non-null    float64
 13  其他人种比例        332 non-null    float64
 14  成年人口          332 non-null    float64
 15  具有高中文凭的比率     332 non-null    float64
 16  大学文凭比例        332 non-null    float64
 17  有工作的人群比例      332 non-null    float64
 18  已知贫困人口        332 non-null    

In [53]:
X = data.iloc[:,1:-3] #除了对象类的列，还有城市编号也是无用的信息，因此我们不把它作为特征矩阵的一部分
X.head()

,面积,总人口,人口密度,白人人口,非裔人口,美洲印第安人人口,亚洲人口,其他人种人口,白人所占比例,非裔所占比例,...,大学文凭比例,有工作的人群比例,已知贫困人口,已知贫困人口的比例,贫困线以下的人的比例,贫困线以下的儿童所占比例,贫困的成年人所占的比例,贫困的老年人所占的比例,是否拥有地铁,点的尺寸
0,0.052,66090.0,1270.961540,63917.0,1702.0,98.0,249.0,124.0,96.712059,2.575276,...,19.631392,4.355859,63628.0,96.274777,13.151443,18.011717,11.009776,12.443812,0.0,250.944411
1,0.014,10626.0,759.000000,7054.0,3496.0,19.0,48.0,9.0,66.384340,32.900433,...,11.243308,2.870315,10529.0,99.087145,32.244278,45.826514,27.385647,25.228976,0.0,185.781260
2,0.022,14991.0,681.409091,14477.0,429.0,35.0,16.0,34.0,96.571276,2.861717,...,17.033819,4.488572,14235.0,94.956974,12.068844,14.036061,10.852090,12.697410,0.0,175.905385
3,0.017,30806.0,1812.117650,29344.0,127.0,46.0,150.0,1139.0,95.254171,0.412257,...,17.278954,4.197800,30337.0,98.477569,7.209019,11.179536,5.536013,6.217047,1.0,319.823487
4,0.018,5836.0,324.222222,5264.0,547.0,14.0,5.0,6.0,90.198766,9.372858,...,14.475999,3.367680,4815.0,82.505140,13.520249,13.022889,11.143211,19.200000,0.0,130.442161


In [54]:
# 标签

y = data.iloc[:,-3:]
y.head()


,c1,c2,c3
0,0.0,0.0,0.0
1,2.0,1.0,0.0
2,0.0,0.0,0.0
3,0.0,2.0,1.0
4,0.0,0.0,0.0


In [55]:
#然后分训练集和测试集

from sklearn.model_selection import train_test_split as TTS
Xtrain, Xtest, Ytrain, Ytest = TTS(X,y,test_size=0.3,random_state=420) # random_state 同样的训练集和测试集

#恢复索引
for i in [Xtrain,Xtest, Ytrain, Ytest]:
    i.index = range(i.shape[0])

Xtrain.head()

,面积,总人口,人口密度,白人人口,非裔人口,美洲印第安人人口,亚洲人口,其他人种人口,白人所占比例,非裔所占比例,...,大学文凭比例,有工作的人群比例,已知贫困人口,已知贫困人口的比例,贫困线以下的人的比例,贫困线以下的儿童所占比例,贫困的成年人所占的比例,贫困的老年人所占的比例,是否拥有地铁,点的尺寸
0,0.041,15682.0,382.487805,15001.0,375.0,125.0,56.0,125.0,95.657442,2.391277,...,12.427492,2.390578,14534.0,92.679505,14.435118,22.300831,13.310056,10.186757,0.0,137.858283
1,0.030,23265.0,775.500000,23127.0,2.0,50.0,41.0,45.0,99.406834,0.008597,...,16.583835,5.166100,22974.0,98.749194,7.704361,8.136567,6.168723,11.040340,0.0,187.881402
2,0.036,61067.0,1696.305560,51991.0,6342.0,109.0,2178.0,447.0,85.137636,10.385314,...,36.643665,14.089892,54230.0,88.804100,28.371750,26.392211,32.458483,13.815301,0.0,305.082769
3,0.031,18185.0,586.612903,17895.0,23.0,211.0,24.0,32.0,98.405279,0.126478,...,19.039803,5.014362,17942.0,98.663734,13.214803,18.758142,12.037542,9.681284,0.0,163.839599
4,0.079,14181.0,179.506329,11962.0,18.0,2167.0,15.0,19.0,84.352302,0.126930,...,17.947917,4.041667,13897.0,97.997320,20.544002,29.073570,19.195368,14.179318,0.0,112.022528


In [59]:
Xtrain.iloc[:, [*range(23), -1]] # 去掉这一列“是否拥有地铁”

,面积,总人口,人口密度,白人人口,非裔人口,美洲印第安人人口,亚洲人口,其他人种人口,白人所占比例,非裔所占比例,...,具有高中文凭的比率,大学文凭比例,有工作的人群比例,已知贫困人口,已知贫困人口的比例,贫困线以下的人的比例,贫困线以下的儿童所占比例,贫困的成年人所占的比例,贫困的老年人所占的比例,点的尺寸
0,0.041,15682.0,382.487805,15001.0,375.0,125.0,56.0,125.0,95.657442,2.391277,...,66.980137,12.427492,2.390578,14534.0,92.679505,14.435118,22.300831,13.310056,10.186757,137.858283
1,0.030,23265.0,775.500000,23127.0,2.0,50.0,41.0,45.0,99.406834,0.008597,...,77.877321,16.583835,5.166100,22974.0,98.749194,7.704361,8.136567,6.168723,11.040340,187.881402
2,0.036,61067.0,1696.305560,51991.0,6342.0,109.0,2178.0,447.0,85.137636,10.385314,...,78.767251,36.643665,14.089892,54230.0,88.804100,28.371750,26.392211,32.458483,13.815301,305.082769
3,0.031,18185.0,586.612903,17895.0,23.0,211.0,24.0,32.0,98.405279,0.126478,...,76.380796,19.039803,5.014362,17942.0,98.663734,13.214803,18.758142,12.037542,9.681284,163.839599
4,0.079,14181.0,179.506329,11962.0,18.0,2167.0,15.0,19.0,84.352302,0.126930,...,73.739583,17.947917,4.041667,13897.0,97.997320,20.544002,29.073570,19.195368,14.179318,112.022528
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
227,0.033,30209.0,915.424242,28966.0,632.0,228.0,269.0,114.0,95.885332,2.092092,...,76.315515,16.234240,3.631343,29106.0,96.348770,14.158593,18.582596,12.255761,13.275687,205.691147
228,0.021,29797.0,1418.904760,29181.0,363.0,58.0,122.0,73.0,97.932678,1.218243,...,70.285018,17.546090,5.504344,27849.0,93.462429,11.634170,15.619022,9.610108,12.160494,269.774823
229,0.054,76559.0,1417.759260,74700.0,1142.0,215.0,197.0,305.0,97.571807,1.491660,...,72.278344,15.925464,2.823278,72840.0,95.142308,6.644701,8.393874,5.177405,8.255788,269.629022
230,0.022,4373.0,198.772727,4072.0,266.0,15.0,6.0,14.0,93.116853,6.082781,...,65.154201,14.250266,2.481390,4160.0,95.129202,25.168269,36.472946,24.414869,15.361139,114.474781


In [60]:
#标准化数据集(只需要对连续性变量标准化，是否拥有地铁这个特征就不是连续性变量)
from sklearn.preprocessing import StandardScaler
ss = StandardScaler().fit(Xtrain.iloc[:, [*range(23), -1]]) # 以训练集为标准的均值和方差

Xtrain_ = Xtrain.copy()
Xtest_ = Xtest.copy()
# 标准化除倒数第二列的所有特征
Xtrain_.iloc[:, [*range(23), -1]] = ss.transform(Xtrain.iloc[:, [*range(23), -1]])
Xtest_.iloc[:, [*range(23), -1]]= ss.transform(Xtest.iloc[:, [*range(23), -1]])

# 输出标准化后的数据
Xtrain_.head()


,面积,总人口,人口密度,白人人口,非裔人口,美洲印第安人人口,亚洲人口,其他人种人口,白人所占比例,非裔所占比例,...,大学文凭比例,有工作的人群比例,已知贫困人口,已知贫困人口的比例,贫困线以下的人的比例,贫困线以下的儿童所占比例,贫困的成年人所占的比例,贫困的老年人所占的比例,是否拥有地铁,点的尺寸
0,0.616975,-0.781589,-0.817515,-0.806262,-0.109249,-0.024001,-0.298458,-0.020924,-0.560519,0.359059,...,-0.879865,-0.848804,-0.824188,-1.455058,0.342460,0.839258,0.428156,-0.582661,0.0,-0.817515
1,-0.141464,-0.374282,-0.347133,-0.350269,-0.486592,-0.330845,-0.368263,-0.341073,0.588691,-0.457448,...,-0.002157,0.773189,-0.351151,0.550676,-1.033914,-1.207832,-0.982892,-0.331241,0.0,-0.347133
2,0.272230,1.656180,0.754945,1.269442,5.927225,-0.089461,9.576664,1.267678,-3.784901,3.098491,...,4.233941,5.988182,1.400655,-2.735690,3.192365,1.430564,4.211672,0.486112,0.0,0.754945
3,-0.072515,-0.647145,-0.573205,-0.643865,-0.465347,0.327847,-0.447376,-0.393097,0.281709,-0.417052,...,0.516478,0.684514,-0.633180,0.522436,0.092918,0.327251,0.176721,-0.731545,0.0,-0.573205
4,3.237036,-0.862212,-1.060456,-0.976797,-0.470405,8.330347,-0.489259,-0.445122,-4.025610,-0.416897,...,0.285901,0.116079,-0.859890,0.302219,1.591667,1.818087,1.591028,0.593332,0.0,-1.060456


In [83]:

#使用逻辑回归探索数据集
from sklearn.linear_model import LogisticRegression as logiR
import pandas as pd
# 循环三次，每次建模的y不同
for i in range(3):
    logi = logiR(solver="newton-cg",max_iter=100**20,multi_class="multinomial").fit(Xtrain_,Ytrain.iloc[:,i].ravel()) # 牛顿法newton-cg   ，声明多分类 multinomial
    print(y.columns[i]) # 输出列名
    print("\tTrain:{}".format(logi.score(Xtrain_,Ytrain.iloc[:,i].ravel()))) # 模型的学习能力
    print("\tTest:{}".format(logi.score(Xtest_,Ytest.iloc[:,i].ravel()))) # 模型的泛化能力
    coeff = pd.DataFrame(logi.coef_).T # 每行是我们的特征，每列    二分类对应一个系数，三分类对应的三个系数 对应3分类（A,H,L），25特征
    if i != 2:  # 三分类
        coeff["mean"] = coeff.iloc[:,:-1].mean(axis=1) # 横向求均值
        coeff["name"] = Xtrain.columns # 名字
        coeff.columns = ["Average","High","Low","mean","name"] # 换名字
        coeff= coeff.sort_values(by="mean",ascending=False).head() # 按照均值mean降序排序。看特征重要性程度
    else: # 二分类
        coeff["name"] = Xtrain.columns # 添加一列
        coeff.columns = ["Coef","name"]
        coeff= coeff.sort_values(by="Coef",ascending=False).head() # 按照均值mean降序排序。看特征重要性程度
    print(coeff.head())
    print("\t")


#结论：第一个字母是平均教育程度的高低，第二个字母是贫困水平的高低，第三个字母是城市还是乡村
#基于这个结论，我们对于图可以分析出什么呢？

c1
	Train:0.9956896551724138
	Test:0.97
     Average      High       Low      mean       name
14  1.274035  2.189743 -3.463778  1.731889  具有高中文凭的比率
15 -0.656186  1.390281 -0.734095  0.367047     大学文凭比例
16 -0.363926  0.899042 -0.535116  0.267558   有工作的人群比例
5   0.168316  0.007686 -0.176003  0.088001   美洲印第安人人口
7   0.015589  0.106681 -0.122271  0.061135     其他人种人口
	
c2
	Train:0.978448275862069
	Test:0.97
     Average      High       Low      mean          name
20  0.248239  1.652407 -1.900646  0.950323  贫困线以下的儿童所占比例
19  0.054842  1.639717 -1.694559  0.847279    贫困线以下的人的比例
21 -0.249106  1.694901 -1.445795  0.722897   贫困的成年人所占的比例
22 -0.105854  0.797378 -0.691524  0.345762   贫困的老年人所占的比例
5   0.256889  0.219229 -0.476118  0.238059      美洲印第安人人口
	
c3
	Train:1.0
	Test:1.0
        Coef    name
23  2.981752  是否拥有地铁
3   0.104570    白人人口
17  0.104304  已知贫困人口
1   0.096218     总人口
8   0.072634  白人所占比例
	


In [76]:
# 单独学习讲解
#使用逻辑回归探索数据集
from sklearn.linear_model import LogisticRegression as logiR
import pandas as pd

logi = logiR(solver="newton-cg",max_iter=100**20,multi_class="multinomial").fit(Xtrain_,Ytrain.iloc[:,0].ravel()) # 牛顿法newton-cg   ，声明多分类 multinomial
print(y.columns[i]) # 输出列名
print("\tTrain:{}".format(logi.score(Xtrain_,Ytrain.iloc[:,i].ravel()))) # 模型的学习能力
print("\tTest:{}".format(logi.score(Xtest_,Ytest.iloc[:,i].ravel()))) # 模型的泛化能力



c3
	Train:0.6767241379310345
	Test:0.71


In [77]:
logi.coef_.shape  #逻辑回归的系数 
# (3, 25)对应3分类（A,H,L），25特征

(3, 25)

In [78]:
coeff = pd.DataFrame(logi.coef_).T # 逻辑回归的系数
coeff # 25行，3列

,0,1,2
0,-0.261786,0.383233,-0.121448
1,-0.012602,-0.061139,0.073742
2,-0.035974,-0.008093,0.044067
3,-0.023190,-0.053499,0.076690
4,0.084845,-0.177490,0.092645
5,0.168316,0.007686,-0.176003
6,0.231448,-0.176674,-0.054774
7,0.015589,0.106681,-0.122271
8,-0.081231,0.069068,0.012164
9,0.027529,-0.018855,-0.008674
